# Практическое задание 3 

# Классификация предложений с использованием BERT

## курс "Математические методы анализа текстов"


### ФИО: АМИНОВ ТИМУР ВЕНЕРОВИЧ

## Введение

### Постановка задачи

В этом задании вы будете классифицировать предложения из медицинских статей на несколько классов (background, objective и т.д.). 
Для того, чтобы улучшить качество решения вам предлагается дообучить предобученную нейросетевую архитектуру BERT.

### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
 - [Pytorch](https://pytorch.org/).
 - [Transformers](https://github.com/huggingface/transformers).
 
### Данные

Скачать данные можно здесь: [ссылка на google диск](https://drive.google.com/file/d/13HlWH8jnmsxqDKrEptxOXQg9kkuQMmGq/view?usp=sharing)

## Часть 1. Подготовка данных

Мы будем работать с предложениями из медицинских статей, разбитых на несколько классов. 

In [0]:
import re
from collections import Counter

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Путь к папке с данными:

In [0]:
DATA_PATH = "/content/drive/My Drive/BERT/sentence_classification_data"

Функция считывания данных:

In [0]:
def read_data(file_name):
    """
    Parameters
    ----------
    file_name : str
        Pubmed sentences file path
        
    Returns
    -------
    text_data : list of str
        List of sentences for algorithm
    
    target_data : list of str
        List of sentence categories
    """
    text_data = []
    target_data = []

    with open(file_name, 'r') as f_input:
        for line in f_input:
            if line.startswith('#') or line == '\n':
                continue
            target, text = line.split('\t')[:2]    

            text_data.append(text)
            target_data.append(target)
    
    return text_data, target_data

Считывание данных:

In [0]:
train_data, train_target = read_data(f'{DATA_PATH}/data_train.txt')
test_data, test_target = read_data(f'{DATA_PATH}/data_test.txt')
dev_data, dev_target = read_data(f'{DATA_PATH}/data_dev.txt')

In [0]:
test_data[10]

'ACTRN12612000642886 .\n'

In [0]:
train_target[20]

'RESULTS'

# Часть 2. Построение бейзлайна (1 балл)

В этой части задания вам необходимо построить бейзлайн модель, с которой вы будете сравнивать ваше решение. В качестве бейзлайна вам предлагается использовать модель логистической регрессии на tf-idf представлениях.

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

Перед тем как подать в модель предложения, необходимо их предобработать:
    
1. привести все предложения к нижнему регистру
2. удалить из предложений все непробельные символы кроме букв, цифр
3. все цифры заменить на нули

Метки ответов необходимо преобразовать из текстового вида в числовой (это можно сделать с помощью LabelEncoder).

Затем необходимо построить tf-idf матрицу по выбранным предложениям (используйте для подсчёта tf-idf только train_data!) и обучить на них модель логистической регрессии. Используйте dev выборку для подбора гиперпараметров модели. Добейтесь того, что на test и dev выборках accuracy будет будет выше 0.8.

In [0]:
for i in range(len(test_data)):
    test_data[i] = "".join([z for d in ' '.join(a for a in test_data[i].split()) for x in d for z in x if z.isalnum() or z ==' ']).replace("  ", " ")
    test_data[i] = re.sub(r'([0-9])', r'0', test_data[i]).lower()

In [0]:
for i in range(len(train_data)):
    train_data[i] = "".join([z for d in ' '.join(a for a in train_data[i].split()) for x in d for z in x if z.isalnum() or z ==' ']).replace("  ", " ")
    train_data[i] = re.sub(r'([0-9])', r'0', train_data[i]).lower()

In [0]:
for i in range(len(dev_data)):
    dev_data[i] = "".join([z for d in ' '.join(a for a in dev_data[i].split()) for x in d for z in x if z.isalnum() or z ==' ']).replace("  ", " ")
    dev_data[i] = re.sub(r'([0-9])', r'0', dev_data[i]).lower()

In [0]:
test_data[10]

'actrn00000000000000 '

In [0]:
le = LabelEncoder()
le.fit(train_target)
train_target = le.transform(train_target)
test_target = le.transform(test_target)
dev_target = le.transform(dev_target)

In [0]:
train_target[20]

4

In [0]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_train = vectorizer.fit_transform(train_data)

In [0]:
X_test = vectorizer.transform(test_data)

In [0]:
X_dev = vectorizer.transform(dev_data)

In [0]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics.scorer import make_scorer
import numpy as np

In [0]:
lr_grid = {
    'C': np.logspace(-2, 2, 10),
}


In [0]:
clf = LogisticRegression(penalty='l1', C=10 , verbose = 10)

clf.fit(X_train , train_target)

C:\Apps\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Apps\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[LibLinear]

C:\Apps\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=10,
                   warm_start=False)

In [0]:
a_scorer = make_scorer(accuracy_score)

In [0]:
a_scorer(clf , X_test, test_target)

0.79444242559227263

In [0]:
a_scorer(clf , X_dev, dev_target)

0.79472556339001799

In [0]:
gs_l2 = GridSearchCV(LogisticRegression(penalty='l2'), lr_grid, scoring=a_scorer, cv=5, n_jobs=4, verbose = 10)
gs_l2.fit(X_train , train_target)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    7.8s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   12.5s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:   22.0s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   31.3s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   52.5s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:  1.8min finished
C:\Apps\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Apps\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=4,
             param_grid={'C': array([  1.00000000e-02,   2.78255940e-02,   7.74263683e-02,
         2.15443469e-01,   5.99484250e-01,   1.66810054e+00,
         4.64158883e+00,   1.29154967e+01,   3.59381366e+01,
         1.00000000e+02])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=make_sco

# успех


In [0]:
a_scorer(gs_l2 , X_test, test_target)

0.80727255676556442

In [0]:
a_scorer(gs_l2 , X_dev, dev_target)

0.80931148900871008

In [0]:
gs_l1 = GridSearchCV(LogisticRegression(penalty='l1'), lr_grid, scoring=a_scorer, cv=5, n_jobs=4 , verbose = 10)
gs_l1.fit(X_train , train_target)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    6.2s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   12.0s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:   27.6s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   39.6s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  1.0min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:  2.3min finished
C:\Apps\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Apps\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l1',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=4,
             param_grid={'C': array([  1.00000000e-02,   2.78255940e-02,   7.74263683e-02,
         2.15443469e-01,   5.99484250e-01,   1.66810054e+00,
         4.64158883e+00,   1.29154967e+01,   3.59381366e+01,
         1.00000000e+02])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=make_sco

In [0]:
a_scorer(gs_l1 , X_test, test_target)

0.79972335212819579

In [0]:
a_scorer(gs_l1 , X_dev, dev_target)

0.80188027098022951

## Часть 3. Задание BERT (4 балла за 3 и 4 части)

Так как обучающих предложений очень мало, попробуем использовать модель BERT, предобученную на большом датасете. Будем использовать библиотеку transformers. Для обучения модели используйте данные до обработки из предыдущего пункта.

In [0]:
train_data, y_train = read_data(f'{DATA_PATH}/data_train.txt')
test_data, y_test = read_data(f'{DATA_PATH}/data_test.txt')
dev_data, y_dev = read_data(f'{DATA_PATH}/data_dev.txt')

In [0]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_dev= le.transform(y_dev)
y_test = le.fit_transform(y_test)

In [0]:
set(y_train)

{0, 1, 2, 3, 4}

In [0]:
BERT_MODEL_NAME = "bert-base-uncased"
NUM_LABELS = len(set(y_train))

In [0]:
!pip install transformers

     |████████████████████████████████| 317kB 6.6MB/s 
     |████████████████████████████████| 1.0MB 47.8MB/s 
     |████████████████████████████████| 860kB 38.5MB/s 
     |████████████████████████████████| 645kB 46.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=91d217fb67213ac23dc7c97913f7f81d536dc4da3221e6668ad3090beb1f2a76
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, WarmupLinearSchedule

import torch
from torch.utils.data import DataLoader, Dataset

Модель BERT работает с специальным форматом данных — все токены из предложения получены с помощью алгоритма BPE. Класс BertTokenizer позволяет получить BPE разбиение для предложения.

In [0]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

100%|██████████| 231508/231508 [00:00<00:00, 931000.53B/s]


In [0]:
d  = tokenizer.tokenize(train_data[0])

In [0]:
type(d)

list

['many',
 'pathogen',
 '##ic',
 'processes',
 'and',
 'diseases',
 'are',
 'the',
 'result',
 'of',
 'an',
 'er',
 '##rone',
 '##ous',
 'activation',
 'of',
 'the',
 'complement',
 'cascade',
 'and',
 'a',
 'number',
 'of',
 'inhibitors',
 'of',
 'complement',
 'have',
 'thus',
 'been',
 'examined',
 'for',
 'anti',
 '-',
 'inflammatory',
 'actions',
 '.']

In [0]:
 ids_review  = tokenizer.convert_tokens_to_ids(d)

In [0]:
 ids_review

[2116,
 26835,
 2594,
 6194,
 1998,
 7870,
 2024,
 1996,
 2765,
 1997,
 2019,
 9413,
 20793,
 3560,
 13791,
 1997,
 1996,
 13711,
 16690,
 1998,
 1037,
 2193,
 1997,
 25456,
 1997,
 13711,
 2031,
 2947,
 2042,
 8920,
 2005,
 3424,
 1011,
 20187,
 4506,
 1012]

В библиотеке transformers есть специальный класс для работы с задачей классификации — BertForSequenceClassification. Воспользуемся им, чтобы задать модель.

In [0]:
bert_model = BertForSequenceClassification.from_pretrained(
    BERT_MODEL_NAME, num_labels=NUM_LABELS
)

bert_model.to('cuda')

Реализуем специальный кастомный датасет для токенизированных с помощью BPE предложений. Каждое предложение должно быть преобразовано в последовательность BPE индексов. Не забудьте, что в начале каждого предложения должен стоять специальный токен [CLS], а в конце должен стоять специальный токен [SEP].

Задайте датасет, используя BertTokenizer:

In [0]:
special_tokens_dict = {'cls_token': '<CLS>' , 
                       'sep_token': '<SEP>' }

tokenizer.add_special_tokens(special_tokens_dict)

0

In [0]:
from tqdm import tqdm

In [0]:
class BertTokenizedDataset(Dataset):
    def __init__(self, tokenizer, text_data, target_data=None, max_length=256):
        """
        Parameters
        ----------
        tokenizer : instance of BertTokenizer
        text_data : list of str
            List of input sentences
        target_data : list of int
            List of input targets
        max_length : int
            Maximum length of input sequence (length in bpe tokens)
        """
        
        #tokenized_review = tokenizer.tokenize(text_data)
        tokenized_review = []
        
        #if len(tokenized_review) > max_length:
         #   tokenized_review = tokenized_review[:max_seq_length]
            
        #ids_review  = tokenizer.convert_tokens_to_ids(tokenized_review)
        #ids_review += padding
        #assert len(ids_review) == max_seq_length
        #print(ids_review)
        #ids_review = torch.tensor(ids_review)
        super(BertTokenizedDataset, self).__init__()
        for sent in tqdm(text_data):
          b = tokenizer.tokenize(sent)
          if len(b) > max_length:
            b = b[:max_length]
          #b.append("[SEP]")
          b = ['[CLS]'] + b + ['[SEP]']
          ids_review  = tokenizer.convert_tokens_to_ids(b)
          tokenized_review.append(torch.LongTensor(ids_review))

        
        #sentiment = target_data # color        
        #list_of_labels = [torch.from_numpy(np.array(sentiment))] 
        self.target_data = target_data
        self.data = tokenized_review

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        if self.target_data is not None:
            return self.data[i], self.target_data[i]
        else:
            return self.data[i]

Получите все датасеты для всех типов данных. 

**Замечание**. После получения есть смысл сохранить все датасеты на диск, т.к. предобработка занимает время.

In [0]:
train_dataset = BertTokenizedDataset(tokenizer, train_data, y_train)
dev_dataset = BertTokenizedDataset(tokenizer, dev_data, y_dev)
test_dataset = BertTokenizedDataset(tokenizer, test_data, y_test)

100%|██████████| 170614/170614 [01:49<00:00, 1560.34it/s]


Используем  класс PadSequences, чтобы задать способ паддинга, работающий с встроенным в pytorch DataLoader.

In [0]:
class PadSequences:
    def __init__(self, use_labels=False):
        self.use_labels = use_labels
    
    def __call__(self, batch):
        """
        Parameters
        ----------
        batch : list of objects or list of (object, label)
            Each object is list of int indexes.
            Each label is int.
        """
        data_label_batch = batch if self.use_labels else [(x, 0) for x in batch]
            
        # Sort the batch in the descending order
        sorted_batch = sorted(data_label_batch, key=lambda x: x[0].shape[0], reverse=True)
        # Get each sequence and pad it
        sequences = [x[0] for x in sorted_batch]
        sequences_padded = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True)
        max_lenght = len(sequences[0])

        # Also need to store the length of each sequence
        # This is later needed in order to unpad the sequences
        lengths = torch.LongTensor([[1] * len(x) + [0] * (max_lenght - len(x)) for x in sequences])
        # Don't forget to grab the labels of the *sorted* batch
        
        if self.use_labels:
            labels = torch.LongTensor([x[1] for x in sorted_batch])
            return sequences_padded, lengths, labels
        else:
            return sequences_padded

Зададим DataLoader для каждого из датасетов:

In [0]:
BATCH_SIZE = 16

In [0]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                              collate_fn=PadSequences(use_labels=True))

dev_dataloader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False,
                              collate_fn=PadSequences(use_labels=True))

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False,
                              collate_fn=PadSequences(use_labels=True))

Заметьте, что модель трансформера обучается по достаточному большому размеру батча (обычно 64), который скорее всего не будет влезать на вашу видеокарту. Поэтому, рекомендуется "накапливать" градиенты за несколько итераций. С помощью параметра ACCUMULATION_STEPS задайте, раз в сколько итераций вам необходимо делать шаг метода оптимизации.

In [0]:
EPOCH_AMOUNT = 2
TRAIN_LENGTH = len(train_dataset)
BATCH_SIZE = 16
ACCUMULATION_STEPS = 4

LR = 2e-5

Посчитайте общее число раз, когда ваш оптимизатор будет делать обновления на основе выбранных значений EPOCH_AMOUNT, BATCH_SIZE, ACCUMULATION_STEPS и  TRAIN_LENGTH. Эта величина нужна для правильного задания параметров оптимизаторов.

In [0]:
train_optimization_step_amount =(TRAIN_LENGTH // (BATCH_SIZE * ACCUMULATION_STEPS)) * EPOCH_AMOUNT

Зададим параметры оптимизаторов. Мы будем использовать специальные оптимизаторы из библиотеки transformers AdamW и WarmupLinearSchedule, обеспечивающие плавный разгон и медленное затухание темпа обучения.

In [0]:
optimizer = AdamW(bert_model.parameters(), lr=LR, correct_bias=False)
scheduler = WarmupLinearSchedule(
    optimizer,
    warmup_steps=train_optimization_step_amount * 0.05,
    t_total=train_optimization_step_amount,
)

Для некоторых групп параметров зададим коэффициенты регуляризации.

In [0]:
param_optimizer = list(bert_model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

## Часть 4. Обучение BERT 

Теперь всё готово к тому, чтобы дообучить BERT на датасете train_dataset!

Используйте dev_dataset для выбора гиперпараметров модели и обучения. Задание будет засчтано на полный балл если на dev_dataset и test_dataset точность будет выше 0.84.

In [0]:
torch.cuda.empty_cache()
!nvidia-smi

Mon Nov 18 16:13:51 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    75W / 149W |    794MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [0]:
import numpy as np

In [0]:
bert_model.train()
history_loss = []
step = 0
num = 0
for epoch in range(EPOCH_AMOUNT):

  for batch in tqdm(train_dataloader , desc = "TRANING "):
    in_ind , lenght , lable_ind = batch
    out = bert_model(in_ind.to(device) , labels=lable_ind.to(device))
    loss, logits = out
    if((num+1) % (ACCUMULATION_STEPS *10) == 0):
      print("epoch {} step {} loss ={} ".format(epoch+1, num , round(loss.item() , 5)))
    loss.backward() 
    history_loss.append(loss.item()) 
    num += 1
    if (num + 1) % ACCUMULATION_STEPS == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            step += 1
    






TRANING :   0%|          | 0/1844 [00:00<?, ?it/s]

TRANING :   0%|          | 1/1844 [00:00<13:15,  2.32it/s]

TRANING :   0%|          | 2/1844 [00:00<14:11,  2.16it/s]

TRANING :   0%|          | 3/1844 [00:01<13:20,  2.30it/s]

TRANING :   0%|          | 4/1844 [00:01<14:38,  2.09it/s]

TRANING :   0%|          | 5/1844 [00:02<13:34,  2.26it/s]

TRANING :   0%|          | 6/1844 [00:02<12:23,  2.47it/s]

TRANING :   0%|          | 7/1844 [00:03<12:46,  2.40it/s]

TRANING :   0%|          | 8/1844 [00:03<13:47,  2.22it/s]

TRANING :   0%|          | 9/1844 [00:04<14:39,  2.09it/s]

TRANING :   1%|          | 10/1844 [00:04<13:50,  2.21it/s]

TRANING :   1%|          | 11/1844 [00:04<13:29,  2.27it/s]

TRANING :   1%|          | 12/1844 [00:05<13:35,  2.25it/s]

TRANING :   1%|          | 13/1844 [00:06<15:28,  1.97it/s]

TRANING :   1%|          | 14/1844 [00:06<14:44,  2.07it/s]

TRANING :   1%|          | 15/1844 [00:06<13:50,  2.20it/s]

TRANING :   1%|          | 16/1844 [00:0

epoch 1 step 39 loss =0.34118 




TRANING :   2%|▏         | 40/1844 [00:19<16:02,  1.87it/s]

TRANING :   2%|▏         | 41/1844 [00:20<15:03,  1.99it/s]

TRANING :   2%|▏         | 42/1844 [00:21<16:22,  1.83it/s]

TRANING :   2%|▏         | 43/1844 [00:21<15:37,  1.92it/s]

TRANING :   2%|▏         | 44/1844 [00:22<15:19,  1.96it/s]

TRANING :   2%|▏         | 45/1844 [00:22<14:16,  2.10it/s]

TRANING :   2%|▏         | 46/1844 [00:23<15:32,  1.93it/s]

TRANING :   3%|▎         | 47/1844 [00:23<16:46,  1.78it/s]

TRANING :   3%|▎         | 48/1844 [00:24<15:44,  1.90it/s]

TRANING :   3%|▎         | 49/1844 [00:24<14:20,  2.09it/s]

TRANING :   3%|▎         | 50/1844 [00:24<14:28,  2.07it/s]

TRANING :   3%|▎         | 51/1844 [00:25<14:20,  2.08it/s]

TRANING :   3%|▎         | 52/1844 [00:26<15:08,  1.97it/s]

TRANING :   3%|▎         | 53/1844 [00:26<14:54,  2.00it/s]

TRANING :   3%|▎         | 54/1844 [00:27<14:52,  2.01it/s]

TRANING :   3%|▎         | 55/1844 [00:27<14:44,  2.02it/s]

TRANING :   3%|▎      

epoch 1 step 79 loss =0.14758 




TRANING :   4%|▍         | 80/1844 [00:39<15:16,  1.92it/s]

TRANING :   4%|▍         | 81/1844 [00:40<13:41,  2.15it/s]

TRANING :   4%|▍         | 82/1844 [00:40<15:04,  1.95it/s]

TRANING :   5%|▍         | 83/1844 [00:41<15:40,  1.87it/s]

TRANING :   5%|▍         | 84/1844 [00:41<14:36,  2.01it/s]

TRANING :   5%|▍         | 85/1844 [00:42<14:41,  2.00it/s]

TRANING :   5%|▍         | 86/1844 [00:42<15:03,  1.95it/s]

TRANING :   5%|▍         | 87/1844 [00:43<15:03,  1.94it/s]

TRANING :   5%|▍         | 88/1844 [00:44<16:35,  1.76it/s]

TRANING :   5%|▍         | 89/1844 [00:44<15:01,  1.95it/s]

TRANING :   5%|▍         | 90/1844 [00:44<13:50,  2.11it/s]

TRANING :   5%|▍         | 91/1844 [00:45<13:39,  2.14it/s]

TRANING :   5%|▍         | 92/1844 [00:46<16:56,  1.72it/s]

TRANING :   5%|▌         | 93/1844 [00:47<19:04,  1.53it/s]

TRANING :   5%|▌         | 94/1844 [00:47<16:44,  1.74it/s]

TRANING :   5%|▌         | 95/1844 [00:48<18:12,  1.60it/s]

TRANING :   5%|▌      

...



TRANING :  95%|█████████▌| 1756/1844 [15:05<00:40,  2.15it/s]

TRANING :  95%|█████████▌| 1757/1844 [15:05<00:40,  2.17it/s]

TRANING :  95%|█████████▌| 1758/1844 [15:06<00:42,  2.04it/s]

TRANING :  95%|█████████▌| 1759/1844 [15:06<00:38,  2.22it/s]

TRANING :  95%|█████████▌| 1760/1844 [15:07<00:39,  2.11it/s]

TRANING :  95%|█████████▌| 1761/1844 [15:07<00:40,  2.07it/s]

TRANING :  96%|█████████▌| 1762/1844 [15:08<00:36,  2.24it/s]

TRANING :  96%|█████████▌| 1763/1844 [15:08<00:37,  2.19it/s]

TRANING :  96%|█████████▌| 1764/1844 [15:09<00:41,  1.93it/s]

TRANING :  96%|█████████▌| 1765/1844 [15:09<00:47,  1.65it/s]

TRANING :  96%|█████████▌| 1766/1844 [15:10<00:49,  1.56it/s]

TRANING :  96%|█████████▌| 1767/1844 [15:11<00:42,  1.81it/s]

TRANING :  96%|█████████▌| 1768/1844 [15:11<00:39,  1.93it/s]

TRANING :  96%|█████████▌| 1769/1844 [15:11<00:36,  2.06it/s]

TRANING :  96%|█████████▌| 1770/1844 [15:12<00:36,  2.02it/s]

TRANING :  96%|█████████▌| 1771/1844 [15:13<00:43,  1

epoch 2 step 3639 loss =0.07626 




TRANING :  97%|█████████▋| 1796/1844 [15:25<00:22,  2.12it/s]

TRANING :  97%|█████████▋| 1797/1844 [15:26<00:20,  2.31it/s]

TRANING :  98%|█████████▊| 1798/1844 [15:26<00:21,  2.10it/s]

TRANING :  98%|█████████▊| 1799/1844 [15:27<00:22,  2.00it/s]

TRANING :  98%|█████████▊| 1800/1844 [15:27<00:20,  2.19it/s]

TRANING :  98%|█████████▊| 1801/1844 [15:28<00:24,  1.77it/s]

TRANING :  98%|█████████▊| 1802/1844 [15:29<00:24,  1.70it/s]

TRANING :  98%|█████████▊| 1803/1844 [15:29<00:24,  1.68it/s]

TRANING :  98%|█████████▊| 1804/1844 [15:30<00:24,  1.62it/s]

TRANING :  98%|█████████▊| 1805/1844 [15:30<00:20,  1.86it/s]

TRANING :  98%|█████████▊| 1806/1844 [15:31<00:20,  1.85it/s]

TRANING :  98%|█████████▊| 1807/1844 [15:31<00:18,  2.06it/s]

TRANING :  98%|█████████▊| 1808/1844 [15:32<00:17,  2.06it/s]

TRANING :  98%|█████████▊| 1809/1844 [15:32<00:15,  2.24it/s]

TRANING :  98%|█████████▊| 1810/1844 [15:32<00:14,  2.27it/s]

TRANING :  98%|█████████▊| 1811/1844 [15:33<00:13,  2

epoch 2 step 3679 loss =0.82373 




TRANING : 100%|█████████▉| 1836/1844 [15:45<00:04,  1.76it/s]

TRANING : 100%|█████████▉| 1837/1844 [15:47<00:05,  1.35it/s]

TRANING : 100%|█████████▉| 1838/1844 [15:47<00:03,  1.52it/s]

TRANING : 100%|█████████▉| 1839/1844 [15:47<00:02,  1.74it/s]

TRANING : 100%|█████████▉| 1840/1844 [15:48<00:02,  1.82it/s]

TRANING : 100%|█████████▉| 1841/1844 [15:49<00:01,  1.70it/s]

TRANING : 100%|█████████▉| 1842/1844 [15:49<00:01,  1.89it/s]

TRANING : 100%|█████████▉| 1843/1844 [15:50<00:00,  1.85it/s]

TRANING : 100%|██████████| 1844/1844 [15:50<00:00,  2.17it/s]



In [0]:
bert_model.eval()
num = 0 
dev_acc = 0
for batch in tqdm(dev_dataloader, desc='DEV'):
    in_ind, length, label_ind = batch
    in_ind, label_ind = in_ind.to(device), label_ind.to(device)
    
    logits = bert_model(in_ind.to(device))   
    logits = logits[0].detach().cpu().numpy()
    label_ind = label_ind.to('cpu').numpy().flatten()


    batch_acc = np.sum(np.argmax(logits, axis=1).flatten() == label_ind) / len(label_ind)
    dev_acc += batch_acc
    num += 1
print('Dev Acc =  {}'.format( round(dev_acc / num) ) )





DEV:   0%|          | 0/1809 [00:00<?, ?it/s]



DEV:   0%|          | 1/1809 [00:00<04:55,  6.11it/s]



DEV:   0%|          | 2/1809 [00:00<04:52,  6.17it/s]



DEV:   0%|          | 3/1809 [00:00<05:27,  5.52it/s]



DEV:   0%|          | 4/1809 [00:00<05:40,  5.31it/s]



DEV:   0%|          | 5/1809 [00:00<05:39,  5.32it/s]



DEV:   0%|          | 6/1809 [00:01<05:41,  5.28it/s]



DEV:   0%|          | 7/1809 [00:01<06:19,  4.75it/s]



DEV:   0%|          | 8/1809 [00:01<05:52,  5.11it/s]



DEV:   0%|          | 9/1809 [00:01<05:54,  5.07it/s]



DEV:   1%|          | 10/1809 [00:01<05:31,  5.43it/s]



DEV:   1%|          | 11/1809 [00:02<05:22,  5.58it/s]



DEV:   1%|          | 12/1809 [00:02<05:06,  5.87it/s]



DEV:   1%|          | 13/1809 [00:02<04:56,  6.06it/s]



DEV:   1%|          | 14/1809 [00:02<04:58,  6.01it/s]



DEV:   1%|          | 15/1809 [00:02<05:25,  5.51it/s]



DEV:   1%|          | 16/1809 [00:02<05:35,  5.35it/s]



DEV:   1%|          | 17/180

Dev Acc =  1.0


In [0]:
print('Dev Acc =  {}'.format( round(dev_acc / 1809 , 5) ) ) 

Dev Acc =  0.86567


In [157]:
bert_model.eval()
num = 0 
test_acc = 0
for batch in tqdm(test_dataloader, desc='TEST'):
    in_ind, length, label_ind = batch
    in_ind, label_ind = in_ind.to(device), label_ind.to(device)

    logits = bert_model(in_ind.to(device))   
    logits = logits[0].detach().cpu().numpy()
    label_ind = label_ind.to('cpu').numpy().flatten()


    batch_acc = np.sum(np.argmax(logits, axis=1).flatten() == label_ind) / len(label_ind)
    test_acc += batch_acc
    num += 1






TEST:   0%|          | 0/10664 [00:00<?, ?it/s]



TEST:   0%|          | 1/10664 [00:00<21:27,  8.28it/s]



TEST:   0%|          | 2/10664 [00:00<23:59,  7.41it/s]



TEST:   0%|          | 3/10664 [00:00<24:09,  7.35it/s]



TEST:   0%|          | 4/10664 [00:00<26:12,  6.78it/s]



TEST:   0%|          | 5/10664 [00:00<25:25,  6.99it/s]



TEST:   0%|          | 6/10664 [00:00<25:49,  6.88it/s]



TEST:   0%|          | 7/10664 [00:01<26:13,  6.77it/s]



TEST:   0%|          | 8/10664 [00:01<25:33,  6.95it/s]



TEST:   0%|          | 9/10664 [00:01<25:01,  7.10it/s]



TEST:   0%|          | 10/10664 [00:01<31:26,  5.65it/s]



TEST:   0%|          | 11/10664 [00:01<32:57,  5.39it/s]



TEST:   0%|          | 12/10664 [00:02<36:45,  4.83it/s]



TEST:   0%|          | 13/10664 [00:02<34:22,  5.16it/s]



TEST:   0%|          | 14/10664 [00:02<32:33,  5.45it/s]



TEST:   0%|          | 15/10664 [00:02<29:25,  6.03it/s]



TEST:   0%|          | 16/10664 [00:02<31:32,  5.63it/

In [5]:
print('Test Acc =  {}'.format( round(test_acc / num , 5) ) )

Test Acc =  0.86178


## Бонусная часть (до 3 баллов)

Улучшите качество (на обеих выборках), используя любые способы (кроме использования дополнительных обучающих данных датасета RCT2000):

* $> 0.86$ — 1 балл 
* $> 0.88$ — 2 балла
* $> 0.9$ — 3 балла